Une fois l'entraînement terminé, le processus d'inférence peut commencer. Il débute en utilisant le jeton spécial "SOS" (Start of Sentence) comme premier jeton d'entrée dans le décodeur.

Voici comment le processus d'inférence se déroule étape par étape :

1. **Encodeur** :
   - La phrase source est d'abord passée à travers l'encodeur pour obtenir une représentation de la séquence source.

2. **Initialisation du décodeur** :
   - Le décodeur est initialisé avec le jeton SOS. À partir de là, il génère des prédictions de mots un par un.

3. **Prédiction des mots** :
   - Le décodeur prend la représentation de l'encodeur et le dernier mot prédit comme entrée pour générer le prochain mot dans la séquence cible. Il utilise l'attention sur la sortie de l'encodeur et l'attention sur ses propres sorties précédentes pour cela.

4. **Échantillonnage de mots** :
   - Lors de l'inférence, plutôt que de choisir simplement le mot avec la probabilité la plus élevée, il est souvent préférable d'échantillonner le mot en fonction des probabilités prédites. Cela peut aider à introduire de la variabilité dans les générations.

5. **Poursuite de la génération** :
   - Le processus de prédiction et d'échantillonnage est répété jusqu'à ce qu'un jeton de fin de phrase (EOS) soit généré ou jusqu'à ce qu'une longueur maximale soit atteinte.

6. **Décodage** :
   - Les jetons prédits sont décodés en mots ou en phrases de la langue cible.

7. **Fin de la séquence** :
   - Une fois qu'un jeton EOS est généré, la génération de la séquence cible est terminée.

8. **Post-traitement** :
   - Le texte généré peut nécessiter un certain post-traitement, comme la suppression des jetons de padding ou d'autres ajustements en fonction des besoins spécifiques de l'application.

Le résultat final est la séquence cible générée à partir de la phrase source donnée en entrée. Ce processus est répété pour chaque nouvelle phrase que vous souhaitez traduire.

In [1]:
import os
import pickle
import torch
import torch.optim as optim
from torch.autograd import Variable
import torchtext
from models import Transformer, create_masks, create_trg_mask
import re
import spacy


In [2]:
class tokener(object):
    def __init__(self, lang):
        d = {"en":"en_core_web_sm", "fr":"fr_core_news_sm"}
        self.ob = spacy.load(d[lang])

    def tokenize(self, sent):
        sent = re.sub(r"[\*\"“”\n\\…\+\-\/\=\(\)‘•:\[\]\|’\!;]", " ", str(sent))
        sent = re.sub(r"\!+", "!", sent)
        sent = re.sub(r"\,+", ",", sent)
        sent = re.sub(r"\?+", "?", sent)
        sent = re.sub(r"[ ]+", " ", sent)
        sent = sent.lower()
        sent = [token.text for token in self.ob.tokenizer(sent) if token.text != " "]
        sent = " ".join(sent)
        return sent

In [3]:

def dum_tokenizer(sent):
    return sent.split()

In [4]:
FR = torchtext.data.Field(tokenize=dum_tokenizer, lower=True, init_token="<sos>", eos_token="<eos>",\
                              batch_first=True)
EN = torchtext.data.Field(tokenize=dum_tokenizer, lower=True, batch_first=True)
train = torchtext.data.TabularDataset(os.path.join("..","data", "df.csv"), format="csv", fields=[("EN", EN), ("FR", FR)])
FR.build_vocab(train)
EN.build_vocab(train)
src_vocab = len(EN.vocab)
trg_vocab = len(FR.vocab)
tokenizer_en = tokener("en")

FileNotFoundError: [Errno 2] No such file or directory: '..\\data\\df.csv'

In [ ]:
net = Transformer(src_vocab=src_vocab, trg_vocab=trg_vocab, d_model=512, num=6, n_heads=8).cuda()
checkpoint=torch.load(os.path.join("..","data" ,"test_model_best_0.pth.tar" ))
net.load_state_dict(checkpoint['state_dict'])
net.eval()

In [ ]:
sent = 'I stayed.'
sent = tokenizer_en.tokenize(sent).split()
sent = [EN.vocab.stoi[tok] for tok in sent]
sent = Variable(torch.LongTensor(sent)).unsqueeze(0)

In [ ]:
trg_init = FR.vocab.stoi["<sos>"]
trg_init = Variable(torch.LongTensor([trg_init])).unsqueeze(0)

In [ ]:
cuda=True
trg = trg_init
src_mask, _ = create_masks(sent, trg_init)
sent = sent.cuda();
src_mask = src_mask.cuda()
trg = trg.cuda()
e_out = net.encoder(sent, src_mask)
translated_word = []; translated_word_idxs = []
for i in range(2, 80):
            trg_mask = create_trg_mask(trg, cuda=cuda)

            trg = trg.cuda(); trg_mask = trg_mask.cuda()

            outputs = net.fc1(net.decoder(trg, e_out, src_mask, trg_mask))
            out_idxs = torch.softmax(outputs, dim=2).max(2)[1]

            trg = torch.cat((trg, out_idxs[:,-1:]), dim=1)

            out_idxs = out_idxs.cpu().numpy()

            translated_word_idxs.append(out_idxs.tolist()[0][-1])
            if translated_word_idxs[-1] == FR.vocab.stoi["<eos>"]:
                break
            translated_word.append(FR.vocab.itos[translated_word_idxs[-1]])

print(" ".join(translated_word))
